# MMMI Oversight Script


**TODO**: Parallel execution.

**TODO**: Reduce NL logging/warning in the notebook.

**TODO**: Option for rolling macro updates

**TODO**: track and display agent histories of consumption and earnings as well as dividend payouts.

**TODO**: `requirements.txt`

**TODO**: Begin automated tests

**TODO**: How do deal with varying 'AdjustPrbs' in quarterly and daily solution steps.

**TODO**: What are the macro population statistics, given our concept of macro agents including institutional investors, etc.

**TODO**: Fix steady state computation

In [1]:
import HARK.ConsumptionSaving.ConsPortfolioModel as cpm
from HARK.Calibration.Income.IncomeTools import (
     sabelhaus_song_var_profile,
)
import hark_portfolio_agents as hpa
import itertools
import logging
import math
from math import exp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

## Portfolio Consumption Model

The AttentionSimulation uses the Portfolio Consumption model from HARK.
The agent's resources unfold according to the following transition equations.

Variable | Equation | Operation | Stage | Utility
-- | -- | -- | -- | --
\Risky | ~Dist | Shock | c | --
\TranShk | ~Dist | Shock | c | --
\PermShk | ~Dist | Shock | c | --
\Rport | \Rport = \PortShare * \Risky + (1 - \PortShare) * R | Update | c | --
b | b_{t} = a_{t-1} \RPort | Update | c | --
p | p_{t}=p_{t-1}\PermShk_{t} | Update | c | --
y | y_{t} = p_{t}\TranShk_{t} | Update | c | --
m | m_{t} = b_{t} + y_{t} | Update | c | --
c | c | Control | c | U(c)
a | a_{t} = m_{t} - c_{t} | Update | portshare | --
\PortShare | \PortShare | Control | portshare | 0


## Initialize the Simulation

In [8]:
import graphviz as gv

dot = gv.Digraph(engine='circo')

dot.node('A', 'HARK Agents')
dot.node('B', 'Broker')
dot.node('M', 'Market')
dot.node('F', 'Financial Model')

dot.edge('A', 'B', label="transact")
dot.edge('B', 'M', label="buy/sell")
dot.edge('M', 'F', label="rate of return")
dot.edge('F', 'A', label="expectations")

dot

The AttentionSimulation class operates the "oversight code" with respect to four other simulation elements.

* HARK Agents - The consumers that earn, consume, and invest in the risky asset. When they have new targets for allocating their assets, they transact with the broker to buy or sell assets. They do this at the current 'market price'.
* Broker - The
*
*

### Initialize agents to start

This is to set up the agent parameters and quantities used in the simulation.

`CRRA` refers to the `\rho` parameter in the constant relative risk aversion utility function used by the agents. Utility is over units of resources consumed.

`aNrmInitStd` is normalized initial market assets, standard deviation. The agents' starting wealth is drawn from a lognormal distribution with this standard deviation.

`pLvlInitMean` and `pLvlInitStd` are the mean and standard deviation of the initial level of permanent income of the agents.

In [3]:
## List of tuples: 
##  ( parameters, (i.e. coefficient of relative risk aversion CRRA)
##    number of agents represented,
##    ( initial risky percent, $$$ in risky asset, $$$ in riskless asset)
agent_classes = [
    {
        'CRRA': 2, #2, The level in cstwMPC paper 1.0
        'AgentCount' : 10
    }] * 20 + \
    [{
        'CRRA': 5, #3, 
        'AgentCount' : 10
    }] * 20 + \
    [{
        'CRRA': 10, #10, 
        'AgentCount' : 10
    }] * 20

# Get empirical data from Sabelhaus and Song
ssvp = sabelhaus_song_var_profile()

# Assume all the agents are 40 for now.
# We will need to make more coherent assumptions about the timing and age of the population later.
# Scaling from annual to quarterly
idx_40 = ssvp['Age'].index(40)

### parameters shared by all agents
agent_parameters = {
    'aNrmInitStd' : 0.0,
    'LivPrb' : [0.98 ** 0.25],
    'PermGroFac': [1.01 ** 0.25],
    'pLvlInitMean' : 1.0, # initial distribution of permanent income
    'pLvlInitStd' : 0.0,
    'Rfree' : 1.0,
    'TranShkStd' : [ssvp['TranShkStd'][idx_40] / 2],  # Adjust non-multiplicative shock to quarterly
    'PermShkStd' : [ssvp['PermShkStd'][idx_40] ** 0.25]
}

## Get agent risky allocations

Use this stock price to determine the initial number of shares of the risky asset held by each
agent:
 - $RAS_{i0} = (HEC_{i0}.RAW)/ RAP_0$
 
 where RAW is Risky Asset Wealth, amount of resources allocated to the risky asset.
 
 RAS is Risky Asset Shares, number of shares the agent has.

In [4]:
class AttentionSimulation():
    """
    Encapsulates the "Oversight Code" functions of the experiment.
    Connects the Agent population model with a Broker, a Market simulation,
    and a FinanceModel of expected share prices.
    
    Parameters
    ----------
    
    agents: [HARK.AgentType]
    
    fm: hpa.FinanceModel
    
    q: int - number of quarters
    
    r: int - runs per quarter
    
    a: float - attention rate (between 0 and 1)
    
    """
    agents = None
    
    broker = None
    
    dollars_per_hark_money_unit = 1500
    
    # Number of days in a quarter / An empirical value based on trading calendars.
    days_per_quarter = 60
    
    # A FinanceModel
    fm = None
    
    # Simulation parameters
    quarters_per_simulation = None # Number of quarters to run total
    
    # Number of market runs to do per quarter
    # Valid values: 1, 2, 3, 4, 5, 6, 10, 12, 15, 20, 30, 60...
    runs_per_quarter = None
    
    # For John's prefered condition: days per quarter = runs per quarter
    # Best if an integer.
    days_per_run = None
    
    ## upping this to make more agents engaged in trade
    attention_rate = None
    
    # for tracking history of the simulation
    history = {}
    
    # dividend_ror -> on financial model
    # dividend_std -> on financial model; on MarketPNL
    
    # sp500_ror = 0.000628 -> on financial model; on MarketPNL
    # sp500_std = 0.011988 -> on financial model; on MarketPNL
    
    def __init__(self, agents, fm, q = 1, r = None, a = None):
        
        self.agents = agents
        self.fm = fm
        
        self.quarters_per_simulation = q
        
        if r is not None:
            self.runs_per_quarter = r
        else:
            self.runs_per_quarter = self.days_per_quarter
        self.days_per_run = self.days_per_quarter / self.runs_per_quarter
        
        # TODO: Make this more variable.
        if a is not None:
            self.attention_rate = a
        else:
            self.attention_rate = 1 / self.runs_per_quarter 
        
        
        # Create the Market wrapper
        market = hpa.MarketPNL()
        self.broker = hpa.Broker(market)

        self.history = {}
        self.history['buy_sell'] = []
        self.history['owned_shares'] = []
        self.history['total_assets'] = []
    
    def attend(self, agent):
        """
        Cause the agent to attend to the financial model.
    
        This will update their expectations of the risky asset.
        They will then adjust their owned risky asset shares to meet their
        target.
    
        Return the delta of risky asset shares ordered through the brokers.
        
        NOTE: This MUTATES the agents with their new target share amounts.
        """
        # Note: this mutates the underlying agent
        agent.assign_parameters(**self.fm.risky_expectations())
            
        d_shares = self.compute_share_demand(agent)
            
        delta_shares = d_shares - agent.shares
            
        # NOTE: This mutates the agent
        agent.shares = d_shares
    
        return delta_shares
    
    def compute_share_demand(self, agent):
        """
        Computes the number of shares an agent _wants_ to own.
    
        This involves:
          - Computing a solution function based on their
            expectations and personal properties
          - Using the solution and the agent's current normalized
            assets to compute a share number
        """
        agent.assign_parameters(AdjustPrb = 1.0)
        agent.solve()
        asset_normalized = agent.state_now['aNrm']

        # ShareFunc takes normalized market assets as argument
        risky_share = agent.solution[0].ShareFuncAdj(
            asset_normalized
        )
    
        # denormalize the risky share. See https://github.com/econ-ark/HARK/issues/986
        risky_asset_wealth = risky_share \
                            * asset_normalized \
                            * agent.state_now['pLvl'] \
                            * self.dollars_per_hark_money_unit 
        
        shares = risky_asset_wealth / self.fm.rap()
    
        if (np.isnan(shares)).any():
            print("ERROR: Agent has nan shares")
    
        return shares
    
    def data(self):
        """
        Returns a Pandas DataFrame of the data from the simulation run.
        """
        data = pd.DataFrame.from_dict({
            't' : range(len(self.fm.prices)),
            'prices' : self.fm.prices,
            'buy' : [None] + [bs[0] for bs in self.broker.buy_sell_history],
            'sell' : [None]  + [bs[1] for bs in self.broker.buy_sell_history],
            'owned' : self.history['owned_shares'],
            'total_assets' : self.history['total_assets'],
            'ror' : [None] + self.fm.ror_list,
            'expected_ror' : self.fm.expected_ror_list,
            'expected_std' : self.fm.expected_std_list,
        })

        return data

    
    def macro_update(self, agent):
        """
        Input: an agent, a FinancialModel, and a Broker
    
        Simulates one "macro" period for the agent (quarterly by assumption).
        For the purposes of the simulation, award the agent dividend income
        but not capital gains on the risky asset.
        """
    
        #agent.assign_parameters(AdjustPrb = 0.0)
        agent.solve()
        
        ## For risky asset gains in the simulated quarter,
        ## use only the dividend.
        true_risky_expectations = {
            "RiskyAvg" : agent.parameters['RiskyAvg'],
            "RiskyStd" : agent.parameters['RiskyStd']   
        }
        
        dividend_risky_params = {
            "RiskyAvg" : 1 + self.fm.dividend_ror,
            "RiskyStd" : self.fm.dividend_std
        }
        
        agent.assign_parameters(**dividend_risky_params)
        
        agent.simulate(sim_periods=1)
        
        ## put back the expectations that include capital gains now
        agent.assign_parameters(**true_risky_expectations)
    
        # Selling off shares if necessary to
        # finance this period's consumption
        asset_level_in_shares = agent.state_now['aLvl'] \
            * self.dollars_per_hark_money_unit / self.fm.rap()

        delta = asset_level_in_shares - agent.shares
        delta[delta > 0] = 0

        agent.shares = agent.shares + delta
        self.broker.transact(delta)

    def report(self):
        data = self.data()

        fig, ax = plt.subplots(
            4, 1,
            sharex='col',
            #sharey='col',
            figsize=(12,16),
        )

        ax[0].plot(data['total_assets'], alpha=0.5, label='total assets')
        ax[0].plot([p * o for (p,o) in zip(data['prices'], data['owned'])], alpha=0.5, label='owned share value')
        ax[0].plot([100 * o for (p,o) in zip(data['prices'], data['owned'])], alpha=0.5, label='owned share quantity * p_0')
        ax[0].legend()

        ax[1].plot(data['buy'], alpha=0.5, label='buy')
        ax[1].plot(data['sell'], alpha=0.5, label='sell')
        ax[1].legend()

        ax[2].plot(data['ror'], alpha=0.5, label='ror')
        ax[2].plot(data['expected_ror'], alpha=0.5, label='expected ror')
        ax[2].legend()

        ax[3].plot(data['prices'], alpha=0.5, label='prices')
        ax[3].legend()
    
        ax[0].set_title("Simulation History")
        ax[0].set_ylabel("Dollars")
        ax[1].set_xlabel("t")
        
        plt.show()

    def simulate(self):
        """
        Workhorse method that runs the simulation.
        """
        seeds = itertools.cycle([2,3,6,0,1,4,5])

        # Initialize share ownership for agents
        for agent in self.agents:
            agent.shares = self.compute_share_demand(agent)

        self.track()

        # Main loop
        for quarter in range(self.quarters_per_simulation):
            print(f"Q-{quarter}")
    
            for agent in self.agents:
                self.macro_update(agent)

            day = 0

            for run in range(self.runs_per_quarter):
                print(f"Q-{quarter}:R-{run}")
        
                # Set to a number for a fixed seed, or None to rotate
                seed = next(seeds)

                for agent in self.agents:
                    if random.random() < self.attention_rate:
                        self.broker.transact(self.attend(agent))

                buy_sell, ror = self.broker.trade(seed)
                print("ror: " + str(ror))
        
                new_run = True
        
                for new_day in range(int(self.days_per_run)):
                    if new_run:
                        new_run = False
                    else:
                        # sloppy
                        # problem is that this should really be nan, nan
                        # putting 0,0 here is a stopgap to make plotting code simpler
                        self.broker.buy_sell_history.append((0,0))
            
                    day = day + 1
                    print(f"Q-{quarter}:D-{day}")
        
                    self.update_agent_wealth_capital_gains(self.fm.rap(), ror)
        
                    self.track()

                    # combine these steps?
                    risky_asset_price = self.fm.add_ror(ror)
                    self.fm.calculate_risky_expectations()

                    print(f"Price end of day {day}: {risky_asset_price}")
                    print("") # line break for presentation


    def track(self):
        """
        Tracks the current state of agent's total assets and owned shares
        """
        tal = sum(
                [agent.state_now['aLvl'].sum()
                 for agent
                 in self.agents]
                ) * self.dollars_per_hark_money_unit
        os = sum([sum(agent.shares) for agent in self.agents])

        print("Total Asset Levels: " + str(tal))
        self.history['owned_shares'].append(os)
        self.history['total_assets'].append(tal)
    
    
    def update_agent_wealth_capital_gains(self, old_share_price, ror):
        """
        For all agents,
        given the old share price
        and a rate of return
    
        update the agent's wealth level to adjust
        for the most recent round of capital gains.
        """

        new_share_price = old_share_price * (1 + ror)

        for agent in self.agents:
            old_raw = agent.shares * old_share_price
            new_raw = agent.shares * new_share_price
    
            delta_aNrm = (new_raw - old_raw) / \
                (self.dollars_per_hark_money_unit * agent.state_now['pLvl'])
    
            # update normalized market assets
            # if agent.state_now['aNrm'] < delta_aNrm:
            #     breakpoint()
        
            agent.state_now['aNrm'] = agent.state_now['aNrm'] + delta_aNrm
        
            if (agent.state_now['aNrm'] < 0).any():
                print(f"ERROR: Agent with CRRA {agent.parameters['CRRA']} has negative aNrm after capital gains update.")
                print("Setting normalize assets and shares to 0.")
                agent.state_now['aNrm'][(agent.state_now['aNrm'] < 0)] = 0.0
                ## TODO: This change in shares needs to be registered with the Broker.
                agent.shares[(agent.state_now['aNrm'] == 0)] = 0

            # update non-normalized market assets
            agent.state_now['aLvl'] = agent.state_now['aNrm']  * agent.state_now['pLvl']

### Activating agents

"At the beginning of each trading day $t + 1$, select a random group of HARK agents with replacement."

**TODO**: HARK AgentType classes represent multiple agents (as many as `AgentCount`). It is difficult to have the agents within the same AgentType act 'separately', though these agents will have _ex ante_ and _ex post_ heterogeneity of some variables. For the purpose of MMMI, we may need to scaffold the agent model to make it easier to have multiple independently acting agents.

"**First**, determine the expected return and standard deviation of the risky asset to be used for decision period $t + 1$."

See `calculate_risky_expectations()`.

"**Second**, using [these expectations] calculate the total number of shares of the risky asset that the selected agents want to buy and sell."

"**Third**, using [total buy and sell shares] as input to the Buy Broker and Sell Broker, run a day of NetLogo trading and determine the end of day price of the risky asset."

See `hpa.run_market`.

"**Fourth**, use the final price to calculate the amount of wealth each (all) HARK agent has allocated to the risky asset."

"Repeat each trading day until the end of the quarter. Then update the economic conditions for each HARK agent (e.g., permanent income level), and repeat [daily loop]."

Accounting functions.

## Run the main loop



In [5]:
def run_simulation(i):
    # Initialize the financial model
    fm = hpa.FinanceModel()

    fm.calculate_risky_expectations()
    agent_parameters.update(fm.risky_expectations())

    # Initialize the population model
    agents = hpa.create_distributed_agents(agent_classes, agent_parameters)
    hpa.init_simulations(agents)


    attsim = AttentionSimulation(agents, fm, q = 1, r = 1)
    attsim.simulate()
    
    return attsim.data()

In [6]:
 #run_simulation(0)

In [7]:
import multiprocessing

inputs = range(2)

pool = multiprocessing.Pool()
records = pool.map(run_simulation, inputs)
pool.close()

/home/sb/projects/econ-ark/HARK/HARK/core.py:871: RuntimeWarning: invalid value encountered in add
  self.history[var_name] = np.empty((self.T_sim, self.AgentCount)) + np.nan
/home/sb/projects/econ-ark/HARK/HARK/core.py:871: RuntimeWarning: invalid value encountered in add
  self.history[var_name] = np.empty((self.T_sim, self.AgentCount)) + np.nan


config_file: ../PNL/macroliquidity.ini
config_local_file: ../PNL/macroliquidity_local.iniconfig_file: ../PNL/macroliquidity.ini

config_local_file: ../PNL/macroliquidity_local.ini
Total Asset Levels: 635937.9704426232
Q-0
Total Asset Levels: 635937.9704426232
Q-0
Q-0:R-0
Q-0:R-0
Buy/Sell Limit: (2916, 0)
Output for S:2,BL:2916,SL:0 already exists.Buy/Sell Limit: (2916, 0)
Output for S:2,BL:2916,SL:0 already exists.
Will use cache.

Will use cache.
ror: 0.02799105073529412ror: 0.02799105073529412
Q-0:D-1

Q-0:D-1
Total Asset Levels: 953559.7304921295
Price end of day 1: 102.7991050735294

Q-0:D-2
Total Asset Levels: 953559.7304921295Total Asset Levels: 980250.8692874662

Price end of day 1: 102.7991050735294Price end of day 2: 105.67656003918539


Q-0:D-2

Q-0:D-3
Total Asset Levels: 980250.8692874662
Price end of day 2: 105.67656003918539Total Asset Levels: 1007689.1211030066


Price end of day 3: 108.63455799277358Q-0:D-3


Q-0:D-4Total Asset Levels: 1007689.1211030066

Price end of d

In [9]:
records